In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import requests as rq
import io

In [3]:
response_X = rq.get('https://github.com/Rohan75/ConvolutionalAttention/blob/master/ConvolutionalAttentionNN/train_x.npy?raw=true')
response_X.raise_for_status()

response_y = rq.get('https://github.com/Rohan75/ConvolutionalAttention/blob/master/ConvolutionalAttentionNN/train_y.npy?raw=true')
response_y.raise_for_status()

X = np.load(io.BytesIO(response_X.content), allow_pickle=True)
y = np.load(io.BytesIO(response_y.content), allow_pickle=True)

In [4]:
input_layer = layers.Input(shape=(200, 4))

conv1 = layers.Conv1D(filters=200, kernel_size=13, activation='relu', padding='same', name='conv1')(input_layer)
pool1 = layers.MaxPool1D(pool_size=6)(conv1)

positions = tf.range(pool1.shape[1])
context = layers.Embedding(input_dim=pool1.shape[1], output_dim=pool1.shape[2])(positions)

contextual_meaning = tf.add(pool1, context)
inputs = layers.Dense(200)(contextual_meaning)

query = layers.Dense(200)(inputs)
value = layers.Dense(200)(inputs)

attention, weights = layers.MultiHeadAttention(num_heads=8, key_dim=32)(query, value, return_attention_scores=True)
attention_inputs = tf.add(inputs, attention)

fcl1 = layers.Dense(256, activation='relu')(attention_inputs)
fcl2 = layers.Dense(256, activation='relu')(fcl1)

flat = layers.Flatten()(fcl2)

output = layers.Dense(1, activation='sigmoid')(flat)

In [5]:
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, validation_split=0.1)

Epoch 1/10
282/282 [==============================] - 18s 58ms/step - loss: 0.6700 - accuracy: 0.5428 - val_loss: 0.1735 - val_accuracy: 0.9460
Epoch 2/10
282/282 [==============================] - 17s 60ms/step - loss: 0.1245 - accuracy: 0.9678 - val_loss: 0.1036 - val_accuracy: 0.9750
Epoch 3/10
282/282 [==============================] - 17s 60ms/step - loss: 0.1069 - accuracy: 0.9725 - val_loss: 0.1119 - val_accuracy: 0.9730
Epoch 4/10
282/282 [==============================] - 17s 60ms/step - loss: 0.1088 - accuracy: 0.9724 - val_loss: 0.1161 - val_accuracy: 0.9700
Epoch 5/10
282/282 [==============================] - 17s 60ms/step - loss: 0.0960 - accuracy: 0.9762 - val_loss: 0.0991 - val_accuracy: 0.9750
Epoch 6/10
282/282 [==============================] - 17s 60ms/step - loss: 0.0967 - accuracy: 0.9749 - val_loss: 0.1038 - val_accuracy: 0.9740
Epoch 7/10
282/282 [==============================] - 17s 61ms/step - loss: 0.1091 - accuracy: 0.9713 - val_loss: 0.1019 - val_accuracy: